# ResQ data assingment

https://github.com/resqclub/data-assignment-2024

##### First just test connection and read database

In [43]:
import sqlite3
import pandas as pd

data_connection = sqlite3.connect("mock_resq.db/mock_resq.db")
resq_database = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", data_connection)
display(resq_database)

,name
0,orders
1,providers
2,users
3,master


In [89]:
providers = pd.read_sql("SELECT * FROM providers", data_connection)
providers.id = providers.id.astype(dtype="str")
providers.defaultOfferType = providers.defaultOfferType.astype(dtype="category")
providers.country = providers.country.astype(dtype="category")
providers.registeredDate = pd.to_datetime(providers.registeredDate)
display(providers)

,id,defaultOfferType,country,registeredDate
0,6563612899329659014,meal,fin,2015-11-24 16:33:35
1,8724554181611615450,meal,fin,2016-01-27 00:08:50
2,946161542061641905,meal,fin,2016-02-01 15:03:27
3,7056903344430959871,meal,fin,2016-02-02 14:56:40
4,3497712091356119302,meal,fin,2016-02-03 13:20:29
...,...,...,...,...
4332,5898387063945077169,meal,fin,2023-11-10 08:22:56
4333,6566678889706300354,meal,fin,2023-11-10 11:24:56
4334,3706913415608167052,meal,fin,2023-11-10 17:56:14
4335,5995496277106943613,meal,fin,2023-11-13 16:01:10


In [90]:
users = pd.read_sql("SELECT * FROM users", data_connection)
users.id = users.id.astype(dtype="str")
users.country = users.country.astype(dtype="category")
users.registeredDate = pd.to_datetime(users.registeredDate)
display(users)

,id,country,registeredDate
0,6244948894430711520,FI,2015-11-24 16:31:10
1,7333245412641704133,FI,2015-11-24 16:31:10
2,7387231492403274666,FI,2015-11-24 16:31:10
3,462254365688400616,FI,2015-11-24 16:31:10
4,1895545503734717224,FI,2015-11-24 16:31:10
...,...,...,...
358361,7196092156622441252,FI,2023-11-15 12:45:55
358362,6883201590444718954,SE,2023-11-15 13:05:59
358363,4955549817670357375,FI,2023-11-15 13:24:56
358364,453099225938434358,FI,2023-11-15 13:42:19


In [91]:
orders = pd.read_sql("SELECT * FROM orders", data_connection)
orders.id = orders.id.astype(dtype="str")
orders.userId = orders.userId.astype(dtype="str")
orders.providerId = orders.providerId.astype(dtype="str")
orders.quantity = orders.quantity.astype(dtype="int")
orders.refunded = orders.refunded.astype(dtype="int")
orders.sales = orders.sales.astype(dtype="int")
orders.currency = orders.currency.astype(dtype="category")
orders.createdAt = pd.to_datetime(orders.createdAt)
display(orders)

,id,createdAt,userId,quantity,refunded,currency,sales,providerId
0,4648711062057701806,2023-08-31 10:14:49,833181563296211638,2,0,eur,1000,3518867990385707647
1,1676056141507951956,2023-03-21 17:04:54,7763311846463275691,1,0,eur,400,6413422964860176913
2,7745602867536251060,2023-07-19 09:48:28,8919282109171104948,2,0,eur,680,123356649204044788
3,7319989469562109720,2023-08-10 12:29:01,5785370845306063462,1,0,eur,1099,7268869293921836511
4,8979946097528312402,2022-10-03 09:55:15,8918527236425591239,1,0,eur,200,7530970657789428790
...,...,...,...,...,...,...,...,...
299966,149803950760879444,2022-09-29 10:40:23,5244759954355981787,1,0,eur,600,3669159159567845583
299967,5087256441085251695,2023-07-22 10:31:59,3656697072752790037,1,0,eur,580,7388976155070169497
299968,9102804762466111812,2022-12-08 12:29:32,792109139296450394,1,0,eur,550,8221933215924621731
299969,1639560206671412519,2023-02-16 16:40:52,3105179374942688281,1,0,eur,590,3993606524249171648


##### Problem 1

Data pipeline for analyst.
(Remember full brief in README file or at: https://github.com/resqclub/data-assignment-2024)

Find providers and users with no orders.  
Join tables together to a single master table.  
Append providers and users with no orders to master table.  
Write table to database.

In [92]:
#Not needed but f*** camelcase
providers = providers.rename(
    columns={"id": "id_provider", \
             "defaultOfferType": "default_offer_type", \
             "country": "country_provider", \
             "registeredDate": "registered_date_provider"}
)

users = users.rename(
    columns={"id": "id_user", \
             "country": "country_user", \
             "registeredDate": "registered_date_user"}
)

orders = orders.rename(
    columns={"id": "id_order", \
             "createdAt": "created_at", \
             "userId": "id_user", \
             "providerId": "id_provider"}
)

In [93]:
#Providers not in orders
order_providers = orders["id_provider"]
providers_not_in_orders = providers[~providers.id_provider.isin(order_providers)]
providers_not_in_orders.count()

id_provider                 773
default_offer_type          773
country_provider            773
registered_date_provider    773
dtype: int64

In [94]:
#Users not in orders
order_users = orders["id_user"]
users_not_in_orders = users[~users.id_user.isin(order_users)]
users_not_in_orders.count()

id_user                 235158
country_user            235158
registered_date_user    235158
dtype: int64

In [95]:
#Merge users and providers to orders and append not found providers and users to master table
merged_orders = pd.merge(orders, users, left_on="id_user", right_on="id_user", suffixes=("_order", "_user"))
merged_orders = pd.merge(merged_orders, providers, left_on='id_provider', right_on='id_provider', suffixes=('_user', '_provider'))

merged_orders = merged_orders.append(providers_not_in_orders)
merged_orders = merged_orders.append(users_not_in_orders)

display(merged_orders)

,id_order,created_at,id_user,quantity,refunded,currency,sales,id_provider,country_user,registered_date_user,default_offer_type,country_provider,registered_date_provider
0,4648711062057701806,2023-08-31 10:14:49,833181563296211638,2.0,0.0,eur,1000.0,3518867990385707647,FI,2019-06-26 13:15:21,meal,fin,2022-08-16 06:46:13
1,4940615340289267908,2023-08-14 16:21:49,833181563296211638,1.0,0.0,eur,450.0,3518867990385707647,FI,2019-06-26 13:15:21,meal,fin,2022-08-16 06:46:13
2,4258992481546199223,2022-12-01 09:48:13,7647820375744941343,2.0,0.0,eur,1210.0,3518867990385707647,FI,2020-08-31 12:06:16,meal,fin,2022-08-16 06:46:13
3,7699578353448148497,2023-09-18 16:28:53,6776401880636478633,1.0,0.0,eur,550.0,3518867990385707647,TZ,2021-03-31 19:09:52,meal,fin,2022-08-16 06:46:13
4,5176865709291146496,2023-04-21 14:47:37,4463273930946732852,2.0,0.0,eur,1100.0,3518867990385707647,FI,2023-01-05 16:40:16,meal,fin,2022-08-16 06:46:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
358359,NaN,NaT,6987761753557822569,NaN,NaN,NaN,NaN,NaN,GB,2023-11-15 12:43:38,NaN,NaN,NaT
358360,NaN,NaT,1822945684526230030,NaN,NaN,NaN,NaN,NaN,EE,2023-11-15 12:44:24,NaN,NaN,NaT
358361,NaN,NaT,7196092156622441252,NaN,NaN,NaN,NaN,NaN,FI,2023-11-15 12:45:55,NaN,NaN,NaT
358364,NaN,NaT,453099225938434358,NaN,NaN,NaN,NaN,NaN,FI,2023-11-15 13:42:19,NaN,NaN,NaT


In [96]:
#Write master table to database
merged_orders.to_sql("master", data_connection, if_exists='replace')

##### Problem 2

Customer lifetime value. (Remember full brief in README file or at: https://github.com/resqclub/data-assignment-2024)

Average Order Size = Total sales / Number of orders  
Customer purchase frequency = Number of orders / Number of unique customers  
Average Customer Value = AOS / CPF  
Average Customer Lifespan (in years?) = Latest order date - First order date  
Customer Lifetime Value = ACL * ACV

In [153]:
#Remove refunded orders sales figures and multiply sales with quantity
merged_orders.loc[merged_orders.refunded == 1, "sales"] = 0
merged_orders["quant_sales"] = merged_orders["quantity"] * merged_orders["sales"]

user_clv_table = merged_orders.groupby("id_user").agg(
    country=pd.NamedAgg(column="country_user", aggfunc="max"),
    order_count=pd.NamedAgg(column="id_order", aggfunc="count"),
    total_sales=pd.NamedAgg(column="quant_sales", aggfunc="sum"), 
    first_order_date=pd.NamedAgg(column="created_at", aggfunc="min"),
    last_order_date=pd.NamedAgg(column="created_at", aggfunc="max")
)

In [165]:
user_clv_table["user_lifetime"] = ((user_clv_table["last_order_date"] - user_clv_table["first_order_date"]).dt.total_seconds()) / 31556926

display(user_clv_table)

,country,order_count,total_sales,first_order_date,last_order_date,user_lifetime_first_order,user_lifetime
id_user,,,,,,,
1000035510875090446,FI,2,1300.0,2023-02-21 12:19:39,2023-07-31 12:43:24,0.438111,0.438111
1000051878408348572,FI,0,0.0,NaT,NaT,NaN,NaN
1000058126066742297,SE,0,0.0,NaT,NaT,NaN,NaN
1000074309035584604,FI,0,0.0,NaT,NaT,NaN,NaN
100008347280964599,FI,0,0.0,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...
999854411983380765,FI,1,400.0,2022-10-06 05:18:22,2022-10-06 05:18:22,0.000000,0.000000
999915069307390810,FI,0,0.0,NaT,NaT,NaN,NaN
999918337642140891,FI,0,0.0,NaT,NaT,NaN,NaN


The CVL calculation should be done atleast based on user country since there are different currencies.  
But below is just the average over all users.

Average Order Size = Total sales / Number of orders  
Customer purchase frequency = Number of orders / Number of unique customers  
Average Customer Value = AOS / CPF  
Customer Lifespan (in years?) = Latest order date - First order date  
Average Customer Lifespan = Total customer lifespan / Number of unique customers  
Customer Lifetime Value = ACL * ACV

In [167]:
average_order_size = user_clv_table.total_sales.sum() / user_clv_table.order_count.sum()
customer_purchase_frequency = user_clv_table.order_count.sum() / len(user_clv_table.index)
average_customer_value = average_order_size / customer_purchase_frequency
average_customer_lifespan = user_clv_table.user_lifetime.sum() / len(user_clv_table.index)
customer_lifetime_value = average_customer_lifespan * average_customer_value

In [168]:
print(customer_lifetime_value)

122.95270619692438
